# Pandas and FAME date ranges

Pandas and FAME represent date ranges differently.  This notebook illustrates converting from FAME range to Pandas range using  Qoma utilities package `qomautils`.

This notebook utilizes FAME HLI (host language interface) function `fame_current_tme()` exposed by the `pyhli` package to obtain current date at various frequencies: current business day, current hour, current minute, ...

The test performed below for a variety of FAME frequencies is:
*  convert the FAME `[]int` range to Pandas `DatetimeIndex`
*  convert the Pandas  `DatetimeIndex` back to FAME `[]int` range
*  verify the FAME input range and output range match

In [1]:
import os
from __future__ import print_function

import pandas as pd
import pyhli 
import qoma_smuggler as qm

The Qoma utility function `open_hli()` opens the FAME environment and prints diagnostic information.

In [2]:
if qm.open_hli()!=0:
    raise

Linux 4.14.33+ (#1 SMP Wed Jun 20 01:15:52 PDT 2018) x86_64
Python 2.7.5 (default, Apr 11 2018, 07:36:10) 
[GCC 4.8.5 20150623 (Red Hat 4.8.5-28)]
NumPy 1.14.5 Pandas 0.23.3 FAME HLI 11.63000 pyhli 0.0.6


Each frequency key in the FAME frequency to Pandas frequency map `pandasFreq` will be tested below. A mutable length-one integer array is provided to the FAME HLI routine `fame_current_time()` to obtain the current date at a specified frequency.

In [3]:
date = [-1]
for freq in qm.PANDAS_FREQ:
    # at each frequency, ask FAME for current date
    status = pyhli.fame_current_time(freq, date)
    now = date[0]
    then = now-4
    
    # FAME range specifies: freq,start,end
    frng_in = [freq,then,now]
    # get Pandas range from FAME range
    prng = qm.to_pandas_range(frng_in)
    # get FAME range from Pandas range
    frng_out = qm.to_fame_range(prng)
    
    # check that the FAME range in matches the FAME range out
    matchString = "match" if frng_in == frng_out else "NO MATCH"
    print("{0}\n{1:10s} FAME in {2} FAME out {3}\nPandas interim\n{4}\n".format(
        qm.type_to_string(freq),
        matchString,
        frng_in,
        frng_out,
        prng
    ))

DATE(MONTHLY)
match      FAME in [129, 2019, 2023] FAME out [129, 2019, 2023]
Pandas interim
DatetimeIndex(['2018-03-31', '2018-04-30', '2018-05-31', '2018-06-30',
               '2018-07-31'],
              dtype='datetime64[ns]', freq='M')

DATE(QUARTERLY)
match      FAME in [162, 671, 675] FAME out [162, 671, 675]
Pandas interim
DatetimeIndex(['2017-09-30', '2017-12-31', '2018-03-31', '2018-06-30',
               '2018-09-30'],
              dtype='datetime64[ns]', freq='Q-DEC')

DATE(MINUTELY)
match      FAME in [227, 20292207, 20292211] FAME out [227, 20292207, 20292211]
Pandas interim
DatetimeIndex(['2018-07-31 19:26:00', '2018-07-31 19:27:00',
               '2018-07-31 19:28:00', '2018-07-31 19:29:00',
               '2018-07-31 19:30:00'],
              dtype='datetime64[ns]', freq='T')

DATE(HOURLY)
match      FAME in [228, 338200, 338204] FAME out [228, 338200, 338204]
Pandas interim
DatetimeIndex(['2018-07-31 15:00:00', '2018-07-31 16:00:00',
               '2018-07-31 17:0

The Qoma utility function `close_hli()` closes the FAME environment.

In [4]:
if qm.close_hli()!=0:
    raise